In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Project dataset/Data
# Load json question file.

import json
from pprint import pprint

with open('OpenEnded_mscoco_val2014_questions.json') as file:
    open_question_json_val = json.load(file)
    
with open('OpenEnded_mscoco_train2014_questions.json') as file:
    open_question_json_train = json.load(file)

/content/drive/MyDrive/Project dataset/Data


In [3]:
# Load json answers file

with open('mscoco_val2014_annotations.json') as file:
    annotation_json_val = json.load(file)
    
with open('mscoco_train2014_annotations.json') as file:
    annotation_json_train = json.load(file)

In [4]:
# Preprocess openended_question_json file
keys_to_remove = ["data_subtype", "data_type", "info", "license", "task_type"]
for key in keys_to_remove:
    open_question_json_val.pop(key, None)
    open_question_json_train.pop(key, None)

In [5]:
import pandas as pd
df_question_val = pd.DataFrame.from_dict(open_question_json_val["questions"], orient = "columns")
df_question_train = pd.DataFrame.from_dict(open_question_json_train["questions"], orient = "columns")

In [6]:
#Remove keys from answers json file.
keys_to_remove = ["data_subtype", "data_type", "info", "license"]
for key in keys_to_remove:
    annotation_json_val.pop(key, None)
    annotation_json_train.pop(key, None)

In [7]:
df_annotation_val = pd.DataFrame.from_dict(annotation_json_val["annotations"], orient = "columns")
df_annotation_train = pd.DataFrame.from_dict(annotation_json_train["annotations"], orient = "columns")

In [8]:
print(df_annotation_val)
print(df_annotation_train)

       question_type multiple_choice_answer  \
0        what is the                   wood   
1             is the                     no   
2               what                kettles   
3         is this an                     no   
4          are there                    yes   
...              ...                    ...   
121507       what is            rubber band   
121508  what kind of                reading   
121509     what time                  10:32   
121510      does the                    yes   
121511      is there                    yes   

                                                  answers  image_id  \
0       [{'answer': 'wood', 'answer_confidence': 'yes'...    350623   
1       [{'answer': 'no', 'answer_confidence': 'yes', ...    350623   
2       [{'answer': 'kettles', 'answer_confidence': 'y...    350623   
3       [{'answer': 'yes', 'answer_confidence': 'maybe...      8647   
4       [{'answer': 'yes', 'answer_confidence': 'yes',...      8647   
...      

In [9]:
df_annotation_val = df_annotation_val.drop(columns = ["answer_type", "answers", "image_id", "question_type"])
df_annotation_train = df_annotation_train.drop(columns = ["answer_type", "answers", "image_id", "question_type"])
print( df_annotation_val.head(5) )
print( df_annotation_train.head(5) )

  multiple_choice_answer  question_id
0                   wood      3506232
1                     no      3506230
2                kettles      3506231
3                     no        86472
4                    yes        86470
  multiple_choice_answer  question_id
0                 curved      4870250
1                    yes      4870251
2                      1      4870252
3                     no       780770
4                  white       780771


In [10]:
data_val_qa = pd.merge(df_question_val, df_annotation_val, how = "outer", on = "question_id")
data_train_qa = pd.merge(df_question_train, df_annotation_train, how = "outer", on = "question_id")

In [11]:
data_val_qa = data_val_qa.drop( columns = ["question_id"])
data_train_qa = data_train_qa.drop( columns = ["question_id"])

In [12]:
print( data_val_qa.count() )
print( data_train_qa.count() )

question                  121512
image_id                  121512
multiple_choice_answer    121512
dtype: int64
question                  248349
image_id                  248349
multiple_choice_answer    248349
dtype: int64


In [13]:
import re

punctuation = []
length = int(len(data_train_qa))
for i in range(length):
    punctuation.append(re.findall(r"[^a-zA-Z0-9 ]", data_train_qa["question"][i]))

In [14]:
myset = set()
for i in range(200):
    for j in range(len(punctuation[i])):
        myset.add(punctuation[i][j])
print(myset)

{',', "'", '?'}


In [15]:
question_punc = []
length = int(len(data_train_qa))
for i in range(length):  
    punctuation = re.findall(r"[']", data_train_qa["question"][i])
    if punctuation:
        question_punc.append([i, data_train_qa["question"][i]])
    
question_punc[0:100]

[[19, "What animal is on the man's lap?"],
 [20, "What's in the man's right hand?"],
 [33, "What color is this person's shirt?"],
 [35, "What color is the man's pants?"],
 [36, "Why is the woman's head blurry?"],
 [40, "What color is the man's shirt?"],
 [54, "What is the theme of the kites' designs?"],
 [59, "Can you see the dog's teeth?"],
 [65, "What color is the man's shirt?"],
 [78, "What is on the man's head?"],
 [90, "What is the around the boy's neck?"],
 [107, "What is the big fruit above the man's head?"],
 [123, "What is in the girl's mouth?"],
 [125, "Is the girl's hair color natural?"],
 [172, "What color are the man's shoes?"],
 [174, "Does the dog like what he's eating?"],
 [181, "What the elephant's tusks made of?"],
 [203, "What is attached to the man's feet?"],
 [221, "What color is the man's jacket?"],
 [225, "What color is the man's shirt?"],
 [268, "Why does the man need the umbrella if it's not raining?"],
 [270, "What color is the mans' jacket?"],
 [378, "What is

In [16]:
dictionary ={
 "'cause": 'because',
 "'s": 'is',
 "'tis": 'it is',
 "'twas": 'it was',
 "Ha'ta": 'even',
 "I'd": 'I had',
 "I'll": 'I shall',
 "I'm": 'I am',
 "I'm'a": 'I am about to',
 "I'm'o": 'I am going to',
 "I've": 'I have',
 "S'e": 'oh yeah',
 "ain't": 'am not',
 "aren't": 'are not',
 "cain't": 'cannot',
 "can't": 'cannot',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "daren't": 'dare not',
 "daresn't": 'dare not',
 "dasn't": 'dare not',
 "didn't": 'did not',
 "doesn't": 'does not',
 "don't": 'do not',
 "e'er": 'ever',
 "everyone's": 'everyone is',
 'finna': 'fixing to',
 'gimme': 'give me',
 "giv'n": 'given',
 "gon't": 'go not',
 'gonna': 'going to',
 'gotta': 'got to',
 "hadn't": 'had not',
 "hasn't": 'is not',
 "haven't": 'have not',
 "he'd": 'he had',
 "he'll": 'he shall',
 "he's": 'he is',
 "he've": 'he have',
 "how'd": 'how did',
 "how'll": 'how will',
 "how're": 'how are',
 "how's": 'how is',
 'howdy': 'how do you do',
 "isn't": 'is not',
 "it'd": 'it would',
 "it'll": 'it shall',
 "it's": 'it is',
 "let's": 'let us',
 "ma'am": 'madam',
 "may've": 'may have',
 "mayn't": 'may not',
 "might've": 'might have',
 "mightn't": 'might not',
 "must've": 'must have',
 "mustn't": 'must not',
 "mustn't've": 'must not have',
 "ne'er": 'never',
 "needn't": 'need not',
 "o'clock": 'of the clock',
 "o'er": 'over',
 "ol'": 'old',
 "oughtn't": 'ought not',
 'rarely': 'cannot',
 "shalln't": 'shall not',
 "shan't": 'shall not',
 "she'd": 'she had',
 "she'll": 'she shall',
 "she's": 'she is',
 "should've": 'should have',
 "shouldn't": 'should not',
 "shouldn't've": 'should not have',
 "so're": 'so are',
 "somebody's": 'somebody is',
 "someone's": 'someone is',
 "something's": 'something is',
 "that'd": 'that would',
 "that'll": 'that shall',
 "that're": 'that are',
 "that's": 'that is',
 "there'd": 'there had',
 "there'll": 'there shall',
 "there're": 'there are',
 "there's": 'there is',
 "these're": 'these are',
 "they'd": 'they had',
 "they'll": 'they shall',
 "they're": 'they are',
 "they've": 'they have',
 "this's": 'this is',
 "those're": 'those are',
 "to've": 'to have',
 "wasn't": 'was not',
 "we'd": 'we had',
 "we'd've": 'we would have',
 "we'll": 'we will',
 "we're": 'we are',
 "we've": 'we have',
 "weren't": 'were not',
 "what'd": 'what did',
 "what'll": 'what shall',
 "what're": 'what are',
 "what's": 'what is',
 "what've": 'what have',
 "when's": 'when is',
 "where'd": 'where did',
 "where're": 'where are',
 "where's": 'where is',
 "where've": 'where have',
 "which's": 'which is',
 "who'd": 'who would',
 "who'd've": 'who would have',
 "who'll": 'who shall',
 "who're": 'who are',
 "who's": 'who is',
 "who've": 'who have',
 "whom'st": 'whom hast',
 "whom'st'd've": 'whom hast had have',
 "why'd": 'why did',
 "why're": 'why are',
 "why's": 'why is',
 "won't": 'will not',
 "would've": 'would have',
 "wouldn't": 'would not',
 "y'all": 'you all',
 "y'all'd've": 'you all would have',
 "you'd": 'you had',
 "you'll": 'you shall',
 "you're": 'you are',
 "you've": 'you have'
}

In [17]:
# We have numbers here, we are going to convert it to english names.
numbers = []
length = int(len(data_train_qa))
for i in range(length):
    no = re.findall(r"[0-9]", data_train_qa["question"][i])
    if no:
        numbers.append(no)

# Get all unique punctuation from Top 200 questions
mysetnumber = set()
for i in range(600):
    for j in range(len(numbers[i])):
        mysetnumber.add(numbers[i][j])
print(mysetnumber)
  
  

{'0', '3', '2', '4', '5', '7', '6', '8', '1', '9'}


In [18]:
number_dictionary = {
    "1" : "one", 
    "2" : "two", 
    "3": "three", 
    "4" : "four", 
    "5": "five",
    "6": "six",
    "7" : "seven",
    "8": "eight", 
    "9":"nine", 
    "0":"zero"
}

In [19]:
# Review the questions:
def mappingWords(questions,dictionary):
    return " ".join([dictionary.get(w,w) for w in questions.split()])

def mappingNumber(questions, number_dictionary):
    return " ".join([number_dictionary.get(w,w) for w in questions.split()])

def review_question(questions):
    questions = questions.lower()
    questions = mappingWords(questions, dictionary)
    questions = mappingNumber(questions, number_dictionary)
    questions = re.sub(r"[^A-Za-z0-9 ]", "", questions)
    return questions

train_question_list = []

validation_question_list = []
for i in range(len(data_train_qa["question"])):
    train_question_list.append(review_question(data_train_qa["question"][i]))
    
for i in range(len(data_val_qa["question"])):
    validation_question_list.append(review_question(data_val_qa["question"][i]))

In [20]:
df_train_questions = pd.DataFrame(train_question_list, columns = ["questions"])
df_val_questions = pd.DataFrame(validation_question_list, columns = ["questions"])

In [21]:
validation = pd.concat([data_val_qa, df_val_questions], axis = 1)
training   = pd.concat([data_train_qa, df_train_questions], axis = 1)

In [22]:
data_val_qa = validation.drop( columns = ["question"])
data_train_qa = training.drop( columns = ["question"])

In [24]:
data_train_qa = data_train_qa.reindex(columns = ["image_id", "questions", "multiple_choice_answer"])
data_val_qa = data_val_qa.reindex(columns = ["image_id", "questions", "multiple_choice_answer"])

In [25]:
%cd /content/drive/MyDrive/VQA/Preprocessed Data

import pickle

open_pickle_val = open('Validation Data QA.pickle', 'wb')
pickle.dump(data_val_qa, open_pickle_val)
open_pickle_val.close()

open_pickle_train = open('Training Data QA.pickle', 'wb')
pickle.dump(data_train_qa, open_pickle_train)
open_pickle_train.close()

/content/drive/MyDrive/VQA/Preprocessed Data


In [26]:
%cd /content/drive/MyDrive/Project dataset/Data/coco
import json

with open('dataset.json') as f:
    data = json.load(f)

In [27]:
data_val_qa.loc[data_val_qa["image_id"] == 391895]
# Here, each cocoid is related to image_id of training and validation data. So we are going to collect cocoid and create a mapping file for it.

,image_id,questions,multiple_choice_answer
107517,391895,is the road paved,no
107518,391895,what type of road structure can be seen in the...,bridge
107519,391895,is the person wearing a helmet,yes


In [28]:
# creating coco_vgg_id_map file(Used later on)

map_list = [ [data["images"][i]["cocoid"], i]  for i in range(len(data["images"]))]

In [30]:
import pandas as pd

datafrm = pd.DataFrame(map_list)

In [31]:
datafrm.to_csv('/content/drive/MyDrive/VQA/Preprocessed Data/coco_vgg_id_map.txt', header = None, index = None, sep=' ', mode = 'a')